In [1]:
from pikepdf import Pdf, PdfImage
import pytesseract

from PIL import Image

import cv2
import numpy as np
from matplotlib import pyplot as plt

from pyzbar.pyzbar import decode

In [91]:
with Pdf.open('2.pdf') as pdf:

    page = pdf.pages[0]
    keyimage = list(page.images.keys())
    rawimage = page.images[keyimage[0]]
    pdfimage = PdfImage(rawimage)
    img = pdfimage.as_pil_image()

if pdfimage.width / pdfimage.height < 0.6:
    rotation_degrees = pytesseract.image_to_osd(img).split('\n')[1][-2:]
    if rotation_degrees != '0':
        img = img.rotate(int(rotation_degrees),expand=True)

In [92]:


cv2_img = np.array(img)

# 灰度图片
gray = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2GRAY)
# 二值化
binary = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35, -5)
cv2.imshow("binary_picture", binary)  # 展示图片
rows, cols = binary.shape
scale = 40
# 自适应获取核值 识别横线
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (cols // scale, 1))
eroded = cv2.erode(binary, kernel, iterations=1)

dilated_col = cv2.dilate(eroded, kernel, iterations=1)
cv2.imshow("excel_horizontal_line", dilated_col)
# cv2.waitKey(0)
# 识别竖线
scale = 20
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, rows // scale))
eroded = cv2.erode(binary, kernel, iterations=1)
dilated_row = cv2.dilate(eroded, kernel, iterations=1)
cv2.imshow("excel_vertical_line", dilated_row)
# cv2.waitKey(0)


# 标识交点
bitwise_and = cv2.bitwise_and(dilated_col, dilated_row)
cv2.imshow("excel_bitwise_and", bitwise_and)
# cv2.waitKey(0)


# 标识表格
merge = cv2.add(dilated_col, dilated_row)
cv2.imshow("entire_excel_contour", merge)
# cv2.waitKey(0)

# 
# 识别黑白图中的白色交叉点，将横纵坐标取出
ys, xs = np.where(bitwise_and > 0)

# print(ys, xs)
# 纵坐标
y_point_arr = []
# 横坐标
x_point_arr = []
# 通过排序，获取跳变的x和y的值，说明是交点，否则交点会有好多像素值值相近，我只取相近值的最后一点
# 这个10的跳变不是固定的，根据不同的图片会有微调，基本上为单元格表格的高度（y坐标跳变）和长度（x坐标跳变）
i = 0
sort_x_point = np.sort(xs)
for i in range(len(sort_x_point) - 1):
    if sort_x_point[i + 1] - sort_x_point[i] > 20:
        x_point_arr.append(sort_x_point[i])
    i = i + 1
x_point_arr.append(sort_x_point[i])  # 要将最后一个点加入

i = 0
sort_y_point = np.sort(ys)
# print(np.sort(ys))
for i in range(len(sort_y_point) - 1):
    if (sort_y_point[i + 1] - sort_y_point[i] > 20):
        y_point_arr.append(sort_y_point[i])
    i = i + 1
# 要将最后一个点加入
y_point_arr.append(sort_y_point[i])
print('y_point_arr', y_point_arr)
print('x_point_arr', x_point_arr)


# 来个范围筛选
for i in range(len(x_point_arr)):
    if x_point_arr[i] > 100:
        break

left, top, right, bottom = x_point_arr[i+1], y_point_arr[1], x_point_arr[i+4], y_point_arr[2]
# cv2.rectangle(raw, (x_point_arr[i+1], y_point_arr[1]), (x_point_arr[i+4], y_point_arr[2]), (0, 0, 255), 2)

y_point_arr [143, 237, 408, 565, 599, 753, 818, 977]
x_point_arr [160, 223, 581, 785, 856, 883, 1044, 1084, 1120, 1200, 1437, 1520, 1754]


In [93]:

img = img.crop((left, top, right, bottom))
# img.show()
text = pytesseract.image_to_string(img, lang='chi_sim')
print(text)

名        称: 淫科国际医疗贯易《上海) 有限公司戌

纳税人识别号;      91510100MA6CWUW1377
地，师线，  起 和疫机汪各生生Ri和0
开户行及账号:  中国银行成都天府大进支行 125315753




In [ ]:
# 输出二维码坐标

from PIL import Image, ImageDraw

with Pdf.open('22.pdf') as pdf:

    page = pdf.pages[0]
    keyimage = list(page.images.keys())
    rawimage = page.images[keyimage[0]]
    pdfimage = PdfImage(rawimage)
    img = pdfimage.as_pil_image()


img1 = img.crop((0, 0, img.size[0] / 4, img.size[1] / 4))

barcodes = decode(img1)

left_top, _, botton_right, _ = barcodes[0].polygon

print(left_top.x, left_top.y, botton_right.x, botton_right.y)

In [28]:
with Pdf.open('1.pdf') as pdf:

    page = pdf.pages[0]
    keyimage = list(page.images.keys())
    rawimage = page.images[keyimage[0]]
    pdfimage = PdfImage(rawimage)
    img = pdfimage.as_pil_image()

zbar_part = img.crop((0, 0, img.size[0] / 4, img.size[1] / 4))
barcodes = decode(zbar_part)
left, top, width, height = barcodes[0].rect

ocr_part = img.crop((left+width, top+height, img.size[0] / 2, img.size[1] / 3))

text = pytesseract.image_to_string(ocr_part)
ocr_part.show()
print(text)

IndexError: list index out of range

In [52]:
img = cv2.imread('1.png')
# 灰度处理
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

edges = cv2.Canny(gray,50,120)

text = pytesseract.image_to_string(gray)
print(text)
# lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, lines=100, minLineLength=20)

# # for x1,y1,x2,y2 in lines:
# #     cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

# img1 = Image.fromarray(edges)
# img1.show()
# 91510100MA6CWUW177


~91510100MA6CWUW177



In [3]:
img= cv2.imread('22.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
decode(imgray)

[]